# pipeline all models 

In [1]:
import numpy as np
import cv2
import sklearn
import pickle

In [2]:
# face detection
face_detector_model = cv2.dnn.readNetFromCaffe('./models/deploy.prototxt.txt'
                                               ,'./models/res10_300x300_ssd_iter_140000.caffemodel')
# feature extraction
face_feature_model = cv2.dnn.readNetFromTorch('./models/openface.nn4.small2.v1.t7')
# face recognition 
face_recogniation_model = pickle.load(open('./models/machinelearning_face_person_identity.pkl',mode ='rb'))
# emotion recogniation model
emotion_recognition_model = pickle.load(open('./models/machinelearning_face_emotion.pkl',mode ='rb'))

C:\Users\tonmo\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\tonmo\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator SVC from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\tonmo\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\tonmo\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid